In [ ]:
# Example 9: pseudo-range (1) (measured every 6 s)
# SPDX-FileCopyrightText: Copyright (C) 2023 Andreas Naber <annappo@web.de>
# SPDX-License-Identifier: GPL-3.0-only

%matplotlib widget 

import json
import gpslib_tutorial as gpslib
import numpy as np
import matplotlib.pyplot as plt
    
GPS_C  = 2.99792458e8
SMP_RATE = 2.048E6    
REC_POS = (4141657.6,604772.9,4796731.9)  # ECEF coordinates

# possible satellites 10,12,13,15,17,22,23,24,25
SAT0 = 24
SAT1 = 25
TOW_0 = 69447                # behind last code phase error

with open('../data/230914_gpsFrames.json','r') as file:
    FRAME_LST=json.load(file)  

satData,satPos,sfLst,towSet = {},{},{},{}

for satNo in [SAT0,SAT1]:
    satData[satNo] = gpslib.SatData(satNo)
    satPos[satNo] = gpslib.SatPos()
    sfLst[satNo] = list(filter(lambda item:item['SAT']==satNo and 'ID' \
                               in item and item['tow']>=TOW_0,FRAME_LST))
    for sf in sfLst[satNo]:
        satData[satNo].readSubframe(sf)        
    towSet[satNo] = set(sf['tow'] for sf in sfLst[satNo])
        
comTowSet = towSet[SAT0] & towSet[SAT1]

dist = {}
smpTime = {}
dstLst = []
for satNo in [SAT0,SAT1]:
    dist[satNo] = []
    smpTime[satNo] = []
    for sf in sfLst[satNo]:
        tow = sf['tow']
        if tow in comTowSet:
            x,y,z,d_st = satPos[satNo].ecefCoord(tow,satData[satNo].ephemData,
                                                 relCorr=False)
            dstLst.append(d_st)
            d = np.sqrt((x-REC_POS[0])**2+(y-REC_POS[1])**2+(z-REC_POS[2])**2)
            dist[satNo].append(d)
            smpTime[satNo].append(sf['ST']/SMP_RATE+d_st)
    dist[satNo] = np.asarray(dist[satNo])            
    smpTime[satNo] = np.asarray(smpTime[satNo])            
            
result = dist[SAT0]-dist[SAT1] - GPS_C*(smpTime[SAT0]-smpTime[SAT1])

mean = np.mean(result)
std  = np.std(result)

fig = plt.figure(figsize=(5,3))
fig.canvas.header_visible = False
plt.hist(result,bins=20,rwidth=0.8)
plt.ylabel('number / bin')
plt.xlabel('difference / m')
plt.tight_layout()
plt.show()
    